In [1]:
import numpy as np
import sys
sys.path.insert(0,'../')
!cd ../../;make;cd python/test_deform_conv/
import caffe
print caffe

NVCC src/caffe/util/deformable_im2col.cu
src/caffe/util/deformable_im2col.cu(397): error: no instance of function template "caffe::deformable_col2im_gpu_kernel" matches the argument list
            argument types are: (int, const float *, const float *, const float *, const int, const int, const int, const int, const int, const int, const int, const int, const int, const int, const int, int, int, int, float *)
          detected during instantiation of "void caffe::deformable_col2im_coord_gpu(const Dtype *, const Dtype *, const Dtype *, int, int, int, int, int, int, int, int, int, int, int, uint32_t, Dtype *) [with Dtype=float]" 
(405): here

src/caffe/util/deformable_im2col.cu(397): error: no instance of function template "caffe::deformable_col2im_gpu_kernel" matches the argument list
            argument types are: (int, const double *, const double *, const double *, const int, const int, const int, const int, const int, const int, const int, const int, const int, const int, const 

In [2]:
proto_str="""
force_backward: true
input: "data"
input_dim: 1
input_dim: 1
input_dim: 4
input_dim: 4
layer{
    name: "offset"
    type: "Convolution"
    bottom: "data"
    top: "offset"
    param{
        lr_mult: 1
        decay_mult: 1
    }
    param{
        lr_mult: 2
        decay_mult: 0
    }
    convolution_param{
        num_output: 8
        kernel_size: 2
        pad: 0
        stride: 2
        weight_filler{
            type: "constant"
            value: 0
        }
        bias_filler{
            type: "constant"
            value: 0
        }
    }
}
layer{
    name: "dconv"
    type: "DeformableConvolution"
#    type: "Convolution"
    bottom: "data"
    bottom: "offset"
    top: "dconv"
    param{
        lr_mult: 1
        decay_mult: 1
    }
    param{
        lr_mult: 2
        decay_mult: 0
    }
    convolution_param{
        num_output: 1
        kernel_size: 2
        pad: 0
        stride: 2
        weight_filler{
            type: "constant"
            value: 1
        }
        bias_filler{
            type: "constant"
            value: 0
        }
    }
    loss_weight: 1
}
"""
proto_file=open('deform_conv.prototxt','w')
proto_file.write(proto_str)
proto_file.close()

In [3]:
caffe.set_mode_gpu()
net = caffe.Net('deform_conv.prototxt', caffe.TEST)

In [4]:
data=net.blobs['data'].data
offset=net.blobs['offset'].data
dconv=net.blobs['dconv'].data
dweight=net.params['dconv'][0].data
dbias=net.params['dconv'][1].data

In [5]:
print 'data.shape:',data.shape
print 'offset.shape:',offset.shape
print 'dconv.shape:',dconv.shape
print 'dweight.shape:',dweight.shape
print 'dbias.shape:',dbias.shape

data.shape: (1, 1, 4, 4)
offset.shape: (1, 8, 2, 2)
dconv.shape: (1, 1, 2, 2)
dweight.shape: (1, 1, 2, 2)
dbias.shape: (1,)


In [6]:
data[:] = np.array([[[ [ 1, 2, 3, 4],
                       [ 4, 3, 2, 1],
                       [ 1, 2, 3, 4],
                       [ 4, 3, 2, 1]]]])

print data
print dweight

[[[[ 1.  2.  3.  4.]
   [ 4.  3.  2.  1.]
   [ 1.  2.  3.  4.]
   [ 4.  3.  2.  1.]]]]
[[[[ 1.  1.]
   [ 1.  1.]]]]


In [7]:
net.forward()

{'dconv': array([[[[ 10.,  10.],
          [ 10.,  10.]]]], dtype=float32)}

In [8]:
print net.blobs['offset'].data

[[[[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]

  [[ 0.  0.]
   [ 0.  0.]]]]


In [9]:
net.backward()

{'data': array([[[[ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.],
          [ 1.,  1.,  1.,  1.]]]], dtype=float32)}

In [10]:
print net.params['dconv'][0].diff

[[[[ 10.  10.]
   [ 10.  10.]]]]


In [11]:
print net.blobs['data'].diff

[[[[ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]
   [ 1.  1.  1.  1.]]]]


In [12]:
print net.blobs['offset'].diff

[[[[ 1.     1.   ]
   [ 1.     1.   ]]

  [[ 1.     1.   ]
   [ 1.     1.   ]]

  [[ 1.     1.   ]
   [ 1.     1.   ]]

  [[ 1.     1.   ]
   [ 1.     1.   ]]

  [[-1.     0.   ]
   [ 0.     0.   ]]

  [[ 1.     0.1  ]
   [ 0.     0.1  ]]

  [[ 0.     0.   ]
   [ 1.875 -1.875]]

  [[-1.     0.1  ]
   [ 0.     0.1  ]]]]
